In [12]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
df = pd.read_csv(r'D:\PROJECTWORSHOP\ICIIT2024\CSV_data\Basic_GLCM_LBP_GIST\BC-15++.csv')
df.head()

In [ ]:
X = df.drop(columns=['class'])
y = df['class']
le = LabelEncoder()
y_encoded = le.fit_transform(y)
X.shape[1], y_encoded.shape[0]

## RandomForest

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train,y_train)

importances_rf =rf_model.feature_importances_
indices_dt = np.argsort(importances_rf)[::-1]
features_dt = X.columns[indices_dt]

for i in range(len(importances_rf)):
    print(f'feature {features_dt[i]:}: {importances_rf[indices_dt[i]]}')

In [ ]:
top_ranking_rf = 86
top_features_rf = features_dt[:top_ranking_rf]

X_train_rf = X_train[top_features_rf]
X_test_rf = X_test[top_features_rf]

scaler = StandardScaler()
X_train_scaled_rf = scaler.fit_transform(X_train_rf)
X_test_scaled_rf = scaler.fit_transform(X_test_rf)
	
top_features_rf

In [ ]:
top_n = 30
top_features = features_dt[:top_n]
top_importances = importances_rf[indices_dt[:top_n]]

plt.figure(figsize=(10, 6))
plt.barh(range(top_n), top_importances[::-1], align='center')
plt.yticks(range(top_n), top_features[::-1])
# plt.xlabel('Feature Importance')
# plt.title('Top 30 Important Features (Random Forest)')
plt.gca().invert_yaxis()
plt.show()

In [18]:
classifiers = {
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=300, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "XGBoost": xgb.XGBClassifier(eval_metric='logloss', random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0),
    "Extra Trees": ExtraTreesClassifier(n_estimators=300, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
}

In [ ]:
X_combined_rf = np.vstack((X_train_scaled_rf, X_test_scaled_rf))
y_combined_rf = np.hstack((y_train, y_test))

results = {}
for name, clf in classifiers.items():
    accuracies = []
    f1_scores = []
    recalls = []
    precisions = []
    
    for _ in tqdm(range(1)):
        combined_df_rf = pd.DataFrame(X_combined_rf)
        combined_df_rf['target'] = y_combined_rf

        shuffled_df_rf = combined_df_rf.sample(frac=1).reset_index(drop=True)

        X_shuffled_rf = shuffled_df_rf.drop(columns=['target']).values
        y_shuffled_rf = shuffled_df_rf['target'].values

        X_train_shuffled_rf = X_shuffled_rf[:len(X_train_scaled_rf)]
        y_train_shuffled_rf = y_shuffled_rf[:len(y_train)]
        X_test_shuffled_rf = X_shuffled_rf[len(X_train_scaled_rf):]
        y_test_shuffled_rf = y_shuffled_rf[len(y_train):]

        clf.fit(X_train_shuffled_rf, y_train_shuffled_rf)
        y_pred = clf.predict(X_test_shuffled_rf)

        accuracies.append(accuracy_score(y_test_shuffled_rf, y_pred))
        f1_scores.append(f1_score(y_test_shuffled_rf, y_pred, average='weighted'))

    min_accuracy = np.min(accuracies)
    max_accuracy = np.max(accuracies)
    avg_accuracy = np.mean(accuracies)
    avg_f1_score = np.max(f1_scores)
      
    results[name] = (min_accuracy, max_accuracy, avg_accuracy, avg_f1_score)
    
    print(f'{name} - Min Accuracy: {min_accuracy:.4f}, Max Accuracy: {max_accuracy:.4f}, Average Accuracy: {avg_accuracy:.4f}, Average F1 Score: {avg_f1_score:.4f}')

In [ ]:
X_train_shuffled_rf.shape

## Desition Trees

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)

dt_model.fit(X_train,y_train)

importances_dt = dt_model.feature_importances_
indices_dt = np.argsort(importances_dt)[::-1]
features_dt = X.columns[indices_dt]

for i in range(len(importances_dt)):
    print(f'feature {features_dt[i]:}: {importances_dt[indices_dt[i]]}')

In [ ]:
top_ranking_dt = 86
top_features_dt = features_dt[:top_ranking_dt]

X_train_dt = X_train[top_features_dt]
X_test_dt = X_test[top_features_dt]

scaler = StandardScaler()
X_train_scaled_dt = scaler.fit_transform(X_train_dt)
X_test_scaled_dt = scaler.fit_transform(X_test_dt)

top_features_dt

In [ ]:
X_combined_dt = np.vstack((X_train_scaled_dt, X_test_scaled_dt))
y_combined_dt = np.hstack((y_train, y_test))

results = {}
for name, clf in classifiers.items():
    accuracies = []
    f1_scores = []
    for _ in tqdm(range(1)):
        combined_df_dt = pd.DataFrame(X_combined_dt)
        combined_df_dt['target'] = y_combined_dt

        shuffled_df_dt = combined_df_dt.sample(frac=1).reset_index(drop=True)

        X_shuffled_dt = shuffled_df_dt.drop(columns=['target']).values
        y_shuffled_dt = shuffled_df_dt['target'].values

        X_train_shuffled_dt = X_shuffled_dt[:len(X_train_scaled_dt)]
        y_train_shuffled_dt = y_shuffled_dt[:len(y_train)]
        X_test_shuffled_dt = X_shuffled_dt[len(X_train_scaled_dt):]
        y_test_shuffled_dt = y_shuffled_dt[len(y_train):]

        clf.fit(X_train_shuffled_dt, y_train_shuffled_dt)
        y_pred = clf.predict(X_test_shuffled_dt)

        accuracies.append(accuracy_score(y_test_shuffled_dt, y_pred))
        f1_scores.append(f1_score(y_test_shuffled_dt, y_pred, average='weighted'))

    min_accuracy = np.min(accuracies)
    max_accuracy = np.max(accuracies)
    avg_accuracy = np.mean(accuracies)
    avg_f1_score = np.mean(f1_scores)
    
    results[name] = (min_accuracy, max_accuracy, avg_accuracy, avg_f1_score)
    print(f'{name} - Min Accuracy: {min_accuracy:.4f}, Max Accuracy: {max_accuracy:.4f}, Average Accuracy: {avg_accuracy:.4f}, Average F1 Score: {avg_f1_score:.4f}')